In [1]:
import os
import azureml
from shutil import copyfile, rmtree
from pathlib import Path
import urllib.request

from azureml.train.estimator import Estimator
from azureml.core import Workspace, Datastore, Experiment, Environment, Model
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core.conda_dependencies import CondaDependencies 
from azureml.core.resource_configuration import ResourceConfiguration

# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.0.83


In [ ]:
## Install Local PIP dependencies. You only need to install once.
!pip install git+https://github.com/NervanaSystems/nlp-architect.git@absa
!pip show spacy    

In [2]:
import spacy
from nlp_architect.models.absa.inference.inference import SentimentInference
from spacy import displacy
from nlp_architect.models.absa.inference.data_types import TermType

#   
#   
#   
# Step 1 - Setup your environment
#   
![title](images/step1.png)

## Initialize workspace

In [3]:
try:
    ws = Workspace.from_config()
    print("Using workspace:",ws.name,"in region", ws.location)
except:
    print('Workspace not found')

Using workspace: absa_space in region southeastasia


## Attach a Compute Cluster

In [4]:
# Choose a name for your CPU cluster
cluster_name = "Bumblebee"

try:
    cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Using compute cluster:', cluster_name)
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D3_V2',
                                                           vm_priority='lowpriority',
                                                           min_nodes=1,
                                                           max_nodes=4)
    cluster = ComputeTarget.create(ws, cluster_name, compute_config)
    cluster.wait_for_completion(show_output=True)

Using compute cluster: Bumblebee


## Attach a Datastore

In [5]:
# Connect to the default datastore
ds = ws.get_default_datastore()

In [6]:
# Download the dataset. You can skip this step if you have done it already.
urllib.request.urlretrieve('http://nlp.stanford.edu/data/glove.840B.300d.zip', '../dataset/glove.840B.300d.zip')

('../dataset/glove.840B.300d.zip', <http.client.HTTPMessage at 0x7f86402634e0>)

In [7]:
# Upload the data to the datastore
ds.upload('../dataset', target_path='clothing_data', overwrite=True, show_progress=True)

Uploading an estimated of 5 files
Uploading ../dataset/clothing-absa-validation.json
Uploading ../dataset/clothing_absa_train.csv
Uploading ../dataset/clothing_absa_train_small.csv
Uploading ../dataset/clothing_automl.xlsx
Uploading ../dataset/glove.840B.300d.zip
Uploaded ../dataset/clothing-absa-validation.json, 1 files out of an estimated total of 5
Uploaded ../dataset/clothing_absa_train_small.csv, 2 files out of an estimated total of 5
Uploaded ../dataset/clothing_automl.xlsx, 3 files out of an estimated total of 5
Uploaded ../dataset/clothing_absa_train.csv, 4 files out of an estimated total of 5
Uploaded ../dataset/glove.840B.300d.zip, 5 files out of an estimated total of 5
Uploaded 5 files


$AZUREML_DATAREFERENCE_bf7d327950d84bf0b3d6ad52c6555ed2

#   
#   
# Step 2 - Experiment with your data & models
#   
![title](images/step2.png)

## Create an Experiment

In [13]:
# Create an experiment or connect if it exists
exp = Experiment(workspace=ws, name='absa')

## Create and run an Estimator

In [14]:
script_params = {
    '--data_folder': ds,
    '--large': 'yes'
}

nlp_est = Estimator(source_directory='../scripts',
                   script_params=script_params,
                   compute_target=cluster,
                   environment_variables = {'NLP_ARCHITECT_BE':'CPU'},
                   entry_script='train.py',
                   pip_packages=['git+https://github.com/NervanaSystems/nlp-architect.git@absa',
                                 'spacy==2.1.8']
)

In [15]:
# Submit and run the estimator
run = exp.submit(nlp_est)

In [16]:
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

In [17]:
# Select a previous run and show the details
run = [r for r in exp.get_runs() if r.id == 'absa_1581283164_547f2062'][0]
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

### Register Model

In [18]:
model = run.register_model(model_name = 'absa', 
                     model_path = 'outputs', 
                     model_framework =  Model.Framework.TENSORFLOW,
                     model_framework_version = '1.13',
                     description = 'Aspect Based Sentiment Analysis - Intel',
                     tags={'area': 'NLP', 'type': 'unsupervised', 'model_author': "INTEL"},
                     resource_configuration=ResourceConfiguration(cpu=1, memory_in_gb=2))

print('version:',model.version)

version: 5


### Test our model On Sample Data 

In [19]:
# Download the model 
# model = Model(ws, 'absa')
Path("../temp").mkdir(parents=True, exist_ok=True)
model.download(exist_ok=True,target_dir="../temp")

'../temp/outputs'

In [20]:
# load the model
c_aspect_lex = '../temp/outputs/generated_aspect_lex.csv'
c_opinion_lex = '../temp/outputs/generated_opinion_lex_reranked.csv' 
inference = SentimentInference(c_aspect_lex, c_opinion_lex)

In [50]:
# Run the predictions
docs = ["Loved the sweater but hated the jeans",
       "Really great outfit, I really like the shirt",
       "The shoes are bad, but perfect blouse",
       "I think this hat is too big for me"]

sentiment_docs = []

for doc_raw in docs:
    sentiment_doc = inference.run(doc=doc_raw)
    sentiment_docs.append(sentiment_doc)

Processing batch 0
Batch 0 Done
Processing batch 0
Batch 0 Done
Processing batch 0
Batch 0 Done
Processing batch 0
Batch 0 Done


In [51]:
# Visualize the predictions on the data
ents = []
for doc in sentiment_docs:    
    doc_viz = {'text':doc._doc_text, 'ents':[]}
    for s in doc._sentences:
        for ev in s._events:
            for e in ev:
                if e._type == TermType.ASPECT:
                    ent = {'start': e._start, 'end': e._start + e._len,'label':str(e._polarity.value), 'text':str(e._text)}
                    if all(kown_e['start'] != ent['start'] for kown_e in ents):
                        ents.append(ent)
                        doc_viz['ents'].append(ent)
    doc_viz['ents'].sort(key=lambda m: m["start"])
    displacy.render(doc_viz, style="ent", options={'colors':{'POS':'#7CFC00', 'NEG':'#FF0000'}}, manual=True, jupyter=True)

AttributeError: 'NoneType' object has no attribute '_doc_text'

#   
#   
#   
# Step 3 -  Deploy your model in Azure

In [23]:
# Load the latest registered model
model = Model(ws, name='absa')
print("Loaded model version:",model.version)

Loaded model version: 5


### Create an environment file

In [24]:
pip = ["azureml-defaults", 
       "azureml-monitoring", 
       "git+https://github.com/NervanaSystems/nlp-architect.git@absa", 
       "spacy==2.1.8"]

myenv = CondaDependencies.create(pip_packages=pip)

Path("../temp").mkdir(parents=True, exist_ok=True)

with open("../temp/myenv.yml","w") as f:
    f.write(myenv.serialize_to_string())
    
deploy_env = Environment.from_conda_specification('absa_env', "../temp/myenv.yml")
deploy_env.environment_variables={'NLP_ARCHITECT_BE': 'CPU'}

### Create an Inference config

In [25]:
inference_config = InferenceConfig(environment=deploy_env, entry_script="../scripts/score.py")

### Deploy to ACI

In [26]:
# Create a deployment config
deploy_config = AciWebservice.deploy_configuration(
                    cpu_cores = model.resource_configuration.cpu, 
                    memory_gb = model.resource_configuration.memory_in_gb,
                    description='Aspect-Based Sentiment Analysis - Intel')

In [27]:
# Deploy the model to an ACI
deployment = Model.deploy(ws, 
                name='absa-metroplex-aci', 
                models = [model], 
                inference_config = inference_config, 
                deployment_config = deploy_config, 
                overwrite = True)

deployment.wait_for_deployment(show_output=True)

Running.....................................................................................................................................................................................
Succeeded
ACI service creation operation finished, operation "Succeeded"


### Get the scoring details

In [36]:
# TODO connect to previous deployment
deployment = AciWebservice(ws, 'absa-metroplex-aci')

In [37]:
print("Scoring endpoint:",deployment.scoring_uri)
print("Test uri:","http://aiml40.azurewebsites.net/?url="+deployment.scoring_uri)

Scoring endpoint: http://102e9872-3580-4e04-9688-8821235626e5.southeastasia.azurecontainer.io/score
Test uri: http://aiml40.azurewebsites.net/?url=http://102e9872-3580-4e04-9688-8821235626e5.southeastasia.azurecontainer.io/score
